# Initial machine
Objective of this is to build a simple method that runs A* and can find some shortest path between two nodes.

In [1]:
import sys
import os
#path = os.path.abspath("datasets")
sys.path.append('../')
import data_readers

import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
from scipy.spatial import distance
import gensim.downloader
import networkx as nx
from transformers import BertTokenizer, BertModel

from urllib.parse import unquote
from sklearn.metrics.pairwise import cosine_similarity
import math

In [2]:
glove_vectors = gensim.downloader.load('glove-twitter-25')

# The links and edges
wikispeedia= nx.read_edgelist('../datasets/wikispeedia_paths-and-graph/links.tsv',
                                    create_using=nx.DiGraph)

In [8]:
model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Function to get embeddings
def get_embedding(text: str):
    temp_str = text.replace('_', ' ')
    temp_str = unquote(temp_str)
    inputs = tokenizer(temp_str, return_tensors="pt")
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

In [9]:
# For the heuristic, I need to create a function for two reasons:
# A) I need to parse the node and make sure I'm getting the title as the value
# B) I need to take care of cases where the word doesn't exist.
# Carlos already kinda took care of that!

def distance_two_words(w1: str, w2: str):
    """Receives a string that was in the wikispeedia dataset, and transforms it as needed to work
    with word2vec.

    Namly, it splits words """


    embedding1 = get_embedding(w1)
    embedding2 = get_embedding(w2)
    similarity = cosine_similarity(embedding1.detach().numpy(), embedding2.detach().numpy())[0][0]
    # Adding absolute, just in case it is needed
    # Similarity is actually 1 - abs(similarity) + 1,
    # As we want closer words to have a smaller distance
    # The last plus one is to indicate that there would be an extra cost to exploring...
    #   not sure if it's needed though...
    similarity = 1 -abs(similarity) + 1
    # print("First word:", w1, ". Second word:", w2, ". GoodDistance:", similarity)
    return similarity

In [10]:
nx.astar_path(wikispeedia, 'Actor', 'Japan', heuristic=distance_two_words)

['Actor', 'Japan']

Method works, but holy shit it's slow af...

The shortest path found tends to be good enough, but it's actually not!

This method is finding a path that is shortest than the one described in the shortest path dictionary... how?

In the printed example, Actor and Japan are neighbors, but in the dataset it says their distance is of 3!

I'll double check the data to see if it's all correct or not

In [13]:
wikispeedia.adj['Actor']

AtlasView({'Charles_II_of_England': {}, 'Drama': {}, 'Film': {}, 'Greece': {}, 'H%C3%A4nsel_und_Gretel': {}, 'Japan': {}, 'Latin': {}, 'Middle_Ages': {}, 'Opera': {}, 'Television': {}, 'The_Marriage_of_Figaro': {}, 'The_Simpsons': {}, 'William_Shakespeare': {}})

And japan is a neighbor... great!

Proof that this shit is wrong follows, reading in the shortest path df and parsing it as indicated

And it's read as well as possible... fuckkkk

In [122]:
shortest_path = np.genfromtxt("../datasets/wikispeedia_paths-and-graph/shortest-path-distance-matrix.txt",
                                  delimiter=1, missing_values=-1, dtype=int)
articles = pd.read_csv('../datasets/wikispeedia_paths-and-graph/articles.tsv', sep='\t', skiprows=12,
                       names=["article_name"])

shortest_path_df = pd.DataFrame(shortest_path, index=articles.values, columns=articles.values)

In [132]:
shortest_path_df[('Actor',)][('Japan',)]

3

We can redo shortest path easily enough with networkX, so if we need it we'll redo it later.

It's a bit of a deviation, but here's the code for that:

In [17]:
shortest_path_dict = dict(nx.all_pairs_shortest_path(wikispeedia))

In [23]:
shortest_path_dict['Actor']

{'Actor': ['Actor'],
 'Charles_II_of_England': ['Actor', 'Charles_II_of_England'],
 'Drama': ['Actor', 'Drama'],
 'Film': ['Actor', 'Film'],
 'Greece': ['Actor', 'Greece'],
 'H%C3%A4nsel_und_Gretel': ['Actor', 'H%C3%A4nsel_und_Gretel'],
 'Japan': ['Actor', 'Japan'],
 'Latin': ['Actor', 'Latin'],
 'Middle_Ages': ['Actor', 'Middle_Ages'],
 'Opera': ['Actor', 'Opera'],
 'Television': ['Actor', 'Television'],
 'The_Marriage_of_Figaro': ['Actor', 'The_Marriage_of_Figaro'],
 'The_Simpsons': ['Actor', 'The_Simpsons'],
 'William_Shakespeare': ['Actor', 'William_Shakespeare'],
 'Africa': ['Actor', 'Charles_II_of_England', 'Africa'],
 'Amsterdam': ['Actor', 'Charles_II_of_England', 'Amsterdam'],
 'Anglicanism': ['Actor', 'Charles_II_of_England', 'Anglicanism'],
 'Anne_of_Great_Britain': ['Actor',
  'Charles_II_of_England',
  'Anne_of_Great_Britain'],
 'BBC': ['Actor', 'Charles_II_of_England', 'BBC'],
 'British_East_India_Company': ['Actor',
  'Charles_II_of_England',
  'British_East_India_Compan

That works, gives different values compared to the dictionary, so we might redo this into a fixed result and store it somewhere.

Now, back to the important part, the AI!

# Alternate implementation
The shortest path method works, but we want to modify this to be a bit more depth first in relation to the exploration of elements.

This is to replicate more or less how humans work, that they do a depth first search at each iteration and are happy as soon as they find a result. We want to replicate this idea as best as possible

So we modify A star to do the following:
- Return a value as soon as the solution target is found
- Make the fitting of the function a bit easier

I just copy and pasted the code from networkX and modified it!

I am not sure at all how the fuck do I cite it though!

In [30]:
from heapq import heappop, heappush
from itertools import count
from networkx.algorithms.shortest_paths.weighted import _weight_function

def modded_astar_path(G: nx.Graph, source: str, target: str, heuristic=None, weight="weight"):
    """Returns a list of nodes in a shortest path between source and target
    using the A* ("A-star") algorithm.

    There may be more than one shortest path.  This returns only one.

    Parameters
    ----------
    G : NetworkX graph

    source : node
       Starting node for path

    target : node
       Ending node for path

    heuristic : function
       A function to evaluate the estimate of the distance
       from the a node to the target.  The function takes
       two nodes arguments and must return a number.
       If the heuristic is inadmissible (if it might
       overestimate the cost of reaching the goal from a node),
       the result may not be a shortest path.
       The algorithm does not support updating heuristic
       values for the same node due to caching the first
       heuristic calculation per node.

    weight : string or function
       If this is a string, then edge weights will be accessed via the
       edge attribute with this key (that is, the weight of the edge
       joining `u` to `v` will be ``G.edges[u, v][weight]``). If no
       such edge attribute exists, the weight of the edge is assumed to
       be one.
       If this is a function, the weight of an edge is the value
       returned by the function. The function must accept exactly three
       positional arguments: the two endpoints of an edge and the
       dictionary of edge attributes for that edge. The function must
       return a number or None to indicate a hidden edge.

    Raises
    ------
    NetworkXNoPath
        If no path exists between source and target.

    Examples
    --------
    >>> G = nx.path_graph(5)
    >>> print(nx.astar_path(G, 0, 4))
    [0, 1, 2, 3, 4]
    >>> G = nx.grid_graph(dim=[3, 3])  # nodes are two-tuples (x,y)
    >>> nx.set_edge_attributes(G, {e: e[1][0] * 2 for e in G.edges()}, "cost")
    >>> def dist(a, b):
    ...     (x1, y1) = a
    ...     (x2, y2) = b
    ...     return ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5
    >>> print(nx.astar_path(G, (0, 0), (2, 2), heuristic=dist, weight="cost"))
    [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2)]

    Notes
    -----
    Edge weight attributes must be numerical.
    Distances are calculated as sums of weighted edges traversed.

    The weight function can be used to hide edges by returning None.
    So ``weight = lambda u, v, d: 1 if d['color']=="red" else None``
    will find the shortest red path.

    See Also
    --------
    shortest_path, dijkstra_path

    """
    if source not in G or target not in G:
        msg = f"Either source {source} or target {target} is not in G"
        raise nx.NodeNotFound(msg)

    if heuristic is None:
        # The default heuristic is h=0 - same as Dijkstra's algorithm
        def heuristic(u, v):
            return 0

    heuristic = distance_two_words

    push = heappush
    pop = heappop
    weight = _weight_function(G, weight)

    g_succ = G._adj  # For speed-up (and works for both directed and undirected graphs)

    # The queue stores priority, node, cost to reach, and parent.
    # Uses Python heapq to keep in priority order.
    # Add a counter to the queue to prevent the underlying heap from
    # attempting to compare the nodes themselves. The hash breaks ties in the
    # priority and is guaranteed unique for all nodes in the graph.
    c = count()
    queue = [(0, next(c), source, 0, None)]

    # Maps enqueued nodes to distance of discovered paths and the
    # computed heuristics to target. We avoid computing the heuristics
    # more than once and inserting the node into the queue too many times.
    enqueued = {}
    # Maps explored nodes to parent closest to the source.
    explored = {}

    while queue:
        # Pop the smallest item from queue.
        _, __, curnode, dist, parent = pop(queue)

        if curnode == target:
            path = [curnode]
            node = parent
            while node is not None:
                path.append(node)
                node = explored[node]
            path.reverse()
            return path, explored

        if curnode in explored:
            # Do not override the parent of starting node
            if explored[curnode] is None:
                continue

            # Skip bad paths that were enqueued before finding a better one
            qcost, h = enqueued[curnode]
            if qcost < dist:
                continue

        explored[curnode] = parent

        for neighbor, w in g_succ[curnode].items():
            # This is the real only place where the code was changed, as I added a check for the neighbor being adjacent
            if neighbor == target:
                explored[neighbor] = curnode
                path = [neighbor]
                node = curnode
                while node is not None:
                    path.append(node)
                    node = explored[node]
                path.reverse()
                return path, explored

            cost = weight(curnode, neighbor, w)
            if cost is None:
                continue
            ncost = dist + cost
            if neighbor in enqueued:
                qcost, h = enqueued[neighbor]
                # if qcost <= ncost, a less costly path from the
                # neighbor to the source was already determined.
                # Therefore, we won't attempt to push this neighbor
                # to the queue
                if qcost <= ncost:
                    continue
            else:
                h = heuristic(neighbor, target)
            enqueued[neighbor] = ncost, h
            push(queue, (ncost + h, next(c), neighbor, ncost, curnode))

    raise nx.NetworkXNoPath(f"Node {target} not reachable from {source}")

In [34]:
path_1, explore_1 = modded_astar_path(wikispeedia, 'Actor', 'Japan', heuristic=distance_two_words)
path_2, explore_2 = modded_astar_path(wikispeedia, 'Actor', 'Josephus', heuristic=distance_two_words)

In [36]:
path_2

['Actor', 'Charles_II_of_England', 'Idolatry', 'Josephus']

In [71]:
from heapq import heappop, heappush
from itertools import count
from networkx.algorithms.shortest_paths.weighted import _weight_function

def only_depth_first_astar_path(G: nx.Graph, source: str, target: str, heuristic=None, weight="weight"):
    """Returns a list of nodes in a shortest path between source and target
    using the A* ("A-star") algorithm.

    There may be more than one shortest path.  This returns only one.

    Only goes depth first, if it finds no nodes it gives up moving onwards

    """
    if source not in G or target not in G:
        msg = f"Either source {source} or target {target} is not in G"
        raise nx.NodeNotFound(msg)

    if heuristic is None:
        # The default heuristic is h=0 - same as Dijkstra's algorithm
        def heuristic(u, v):
            return 0

    heuristic = distance_two_words

    push = heappush
    pop = heappop
    weight = _weight_function(G, weight)

    g_succ = G._adj  # For speed-up (and works for both directed and undirected graphs)

    # The queue stores priority, node, cost to reach, and parent.
    # Uses Python heapq to keep in priority order.
    # Add a counter to the queue to prevent the underlying heap from
    # attempting to compare the nodes themselves. The hash breaks ties in the
    # priority and is guaranteed unique for all nodes in the graph.
    c = count()

    # Maps explored nodes to parent closest to the source.
    explored = {}

    curnode = source
    parent = None

    while curnode is not None:

        explored[curnode] = parent

        if curnode == target:
            path = [curnode]
            node = parent
            while node is not None:
                path.append(node)
                node = explored[node]
            path.reverse()
            return path, explored

        # Here I explore all the nodes, find the cost and decide only the smallest one
        adjacent_nodes_and_weight = []

        for neighbor, w in g_succ[curnode].items():
            # This is the real only place where the code was changed, as I added a check for the neighbor being adjacent
            if neighbor == target:
                explored[neighbor] = curnode
                path = [neighbor]
                node = curnode
                while node is not None:
                    path.append(node)
                    node = explored[node]
                path.reverse()
                return path, explored

            h = heuristic(neighbor, target)

            adjacent_nodes_and_weight.append((neighbor, h))

        adjacent_nodes_and_weight.sort(key=lambda x: x[1])

        # Now, pick the lowest value that hasn't been explored yet
        # It does mean we could get in a loop, but that's okay!
        parent = curnode
        curnode = None
        #print(parent)

        for node, val in adjacent_nodes_and_weight:
            if not node in explored:
                curnode = node
                break

    raise nx.NetworkXNoPath(f"Node {target} not reachable from {source}")

In [70]:
alt_path_1, alt_explore_1 = only_depth_first_astar_path(wikispeedia, 'Actor', 'Japan', heuristic=distance_two_words)
alt_path_2, alt_explore_2 = only_depth_first_astar_path(wikispeedia, 'Actor', 'Josephus', heuristic=distance_two_words)

Actor
William_Shakespeare
Macbeth
Henry_James
Charles_Darwin
Isaac_Newton
Johannes_Kepler
Aristotle
Rome
Jerusalem
Hebrew_language
Israel
Tel_Aviv
Mesopotamia
Babylonia
Assyria
Egypt
Islam


They both work, which is good.

I can do some comparisons later of the explored of the two, but fuck it this is good enough!